In [ ]:
!fusermount -u /content/drive 2>/dev/null || True
!rm -rf /content/drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


/bin/bash: line 1: True: command not found
Mounted at /content/drive


In [ ]:
!pip install torch torchvision tqdm --quiet
import torch, torch.nn as nn, torch.nn.functional as F, torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import numpy as np, matplotlib.pyplot as plt, os
from sklearn.manifold import TSNE
from scipy.stats import pearsonr
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from google.colab import drive
drive.mount('/content/drive')
os.makedirs("/content/drive/MyDrive/KD_Experiments", exist_ok=True)


Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset
from torchvision.transforms import ToTensor

dataset = load_dataset("Maysee/tiny-imagenet", split="train")
valset  = load_dataset("Maysee/tiny-imagenet", split="valid")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-1359597a978bc4(…):   0%|          | 0.00/146M [00:00<?, ?B/s]

data/valid-00000-of-00001-70d52db3c749a9(…):   0%|          | 0.00/14.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import shutil, os, random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------- 1. Load Tiny-ImageNet ----------
dataset = load_dataset("Maysee/tiny-imagenet")

# ---------- 2. Downsample to 10 000 total ----------
def subsample_split(split, keep=10000):
    n = len(dataset[split])
    idx = random.sample(range(n), min(keep, n))
    return dataset[split].select(idx)

train_subset = subsample_split("train", keep=9000)
val_subset   = subsample_split("valid", keep=1000)
print(f"Using {len(train_subset)} train and {len(val_subset)} val samples.")

# ---------- 3. Immediately delete the heavy cached data ----------
cache_dir = os.path.expanduser("~/.cache/huggingface/datasets")
if os.path.exists(cache_dir):
    print("Removing cached Hugging Face dataset shards to free disk…")
    try:
        shutil.rmtree(cache_dir)
    except Exception as e:
        print("Partial cleanup:", e)

# ---------- 4. Define transforms ----------
transform_train = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])
transform_val = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])

def transform_example(example, transform):
    example["pixel_values"] = transform(example["image"])
    return example

train_hf = train_subset.map(lambda e: transform_example(e, transform_train))
val_hf   = val_subset.map(lambda e: transform_example(e, transform_val))

# ---------- 5. Wrap into PyTorch Datasets ----------
class HFDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset):
        self.ds = hf_dataset
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, idx):
        item = self.ds[idx]
        return item["pixel_values"], int(item["label"])

trainset = HFDataset(train_hf)
valset   = HFDataset(val_hf)

# ---------- 6. Memory-safe DataLoaders ----------
train_loader = DataLoader(trainset, batch_size=32, shuffle=True,
                          num_workers=0, pin_memory=True)
val_loader   = DataLoader(valset, batch_size=32, shuffle=False,
                          num_workers=0, pin_memory=True)

print(f"Train={len(trainset)}, Val={len(valset)} (subset Tiny-ImageNet loaded)")
torch.cuda.empty_cache()


Device: cuda
Using 9000 train and 1000 val samples.
Removing cached Hugging Face dataset shards to free disk…


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Train=9000, Val=1000 (subset Tiny-ImageNet loaded)


In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import models, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------- CONFIG ----------
BATCH_SIZE = 64
EPOCHS = 15
LR = 1e-3
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
T = 2.0          # KD temperature
ALPHA = 0.3      # KD weight for teacher matching

# ---------- Dataset Wrapper ----------
class HFDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset):
        self.ds = hf_dataset
        self.normalize = transforms.Normalize(mean=[0.485,0.456,0.406],
                                              std=[0.229,0.224,0.225])
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, idx):
        item = self.ds[idx]
        x = item["pixel_values"]
        # ensure tensor conversion
        if isinstance(x, list):
            x = torch.tensor(np.array(x), dtype=torch.float32)
        elif isinstance(x, np.ndarray):
            x = torch.tensor(x, dtype=torch.float32)
        x = x.permute(2,0,1) if x.ndim == 3 and x.shape[0] != 3 else x
        x = self.normalize(x/255.0 if x.max() > 1 else x)
        y = int(item["label"])
        return x, y

# ---------- Build DataLoaders ----------
trainset = HFDataset(train_hf)
valset   = HFDataset(val_hf)

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader   = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
print(f"Train={len(trainset)}, Val={len(valset)}")

# ---------- Teacher ----------
num_classes = len(set(train_hf['label']))
teacher = models.vgg16(weights='IMAGENET1K_V1')
teacher.classifier[6] = nn.Linear(4096, num_classes)
teacher = teacher.to(device)
teacher.eval()
print("Teacher (VGG16 pretrained) loaded with classifier head =", num_classes)

# ---------- KD Loss ----------
def kd_loss(student_logits, teacher_logits, labels, T=T, alpha=ALPHA):
    """Hinton KD loss with temperature and cross-entropy mix."""
    soft_t = F.log_softmax(student_logits / T, dim=1)
    soft_s = F.softmax(teacher_logits / T, dim=1)
    kd = F.kl_div(soft_t, soft_s, reduction='batchmean') * (T * T)
    ce = F.cross_entropy(student_logits, labels)
    return alpha * kd + (1 - alpha) * ce

# ---------- Metrics ----------
def topk_acc(output, target, topk=(1,5)):
    """Compute top-k accuracies."""
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, dim=1)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size).item())
    return res

def evaluate(model, loader):
    model.eval(); top1_sum=top5_sum=total=0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            top1, top5 = topk_acc(out, y)
            top1_sum += top1 * y.size(0)
            top5_sum += top5 * y.size(0)
            total += y.size(0)
    return top1_sum / total, top5_sum / total

# ---------- Train Functions ----------
def train_epoch_hard(model, loader, optimizer):
    model.train(); total, correct, loss_sum = 0, 0, 0
    for x, y in tqdm(loader, leave=False):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = F.cross_entropy(out, y)
        loss.backward(); optimizer.step()
        loss_sum += loss.item() * y.size(0)
        _, pred = out.max(1)
        correct += pred.eq(y).sum().item(); total += y.size(0)
    return loss_sum/total, 100*correct/total

def train_epoch_soft(student, teacher, loader, optimizer):
    student.train(); total, correct, loss_sum = 0, 0, 0
    for x, y in tqdm(loader, leave=False):
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            t_logits = teacher(x)
        s_logits = student(x)
        loss = kd_loss(s_logits, t_logits, y)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        loss_sum += loss.item() * y.size(0)
        _, pred = s_logits.max(1)
        correct += pred.eq(y).sum().item(); total += y.size(0)
    return loss_sum/total, 100*correct/total

# ---------- Run Training ----------
def run_training(method="hard"):
    # student starts from scratch (no pretrained weights)
    student = models.vgg11(num_classes=num_classes).to(device)
    opt = torch.optim.SGD(student.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    print(f"\n=== {method.upper()} training start ===")
    for ep in range(1, EPOCHS + 1):
        if method == "soft":
            train_loss, train_acc = train_epoch_soft(student, teacher, train_loader, opt)
        else:
            train_loss, train_acc = train_epoch_hard(student, train_loader, opt)
        val_top1, val_top5 = evaluate(student, val_loader)
        print(f"Epoch {ep:02d}: TrainAcc={train_acc:.2f}% | ValTop1={val_top1:.2f}% | ValTop5={val_top5:.2f}%")
    print(f"=== {method.upper()} training complete ===")
    return student

# ---------- Execute ----------
student_hard = run_training("hard")
student_soft = run_training("soft")


Device: cuda
Train=9000, Val=1000
Teacher (VGG16 pretrained) loaded with classifier head = 200

=== HARD training start ===


OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 276.12 MiB is free. Process 4107 has 14.47 GiB memory in use. Of the allocated memory 13.17 GiB is allocated by PyTorch, and 1.16 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from google.colab import drive
import torch, os

# Mount Drive
drive.mount('/content/drive')

# Create save directory
save_dir = "/content/drive/MyDrive/KD_TinyImageNet_Models"
os.makedirs(save_dir, exist_ok=True)

# Save both models
hard_path  = os.path.join(save_dir, "VGG11_HARD_KD.pth")
soft_path  = os.path.join(save_dir, "VGG11_SOFT_KD.pth")

torch.save(student_hard.state_dict(), hard_path)
torch.save(student_soft.state_dict(), soft_path)

print(f"✅ Models saved permanently to Google Drive:")
print(f"  • Hard KD → {hard_path}")
print(f"  • Soft KD → {soft_path}")


In [ ]:
def correlation_matrix(model, loader, n_batches=10):
    model.eval(); feats=[]
    with torch.no_grad():
        for i,(x,_) in enumerate(loader):
            if i>=n_batches: break
            x=x.to(device)
            feats.append(model(x).detach().cpu().numpy())
    feats=np.concatenate(feats,axis=0)
    return np.corrcoef(feats.T)

print("\n--- Final Evaluation ---")
for label,model in [("Hard",student_hard),("Soft",student_soft)]:
    top1,top5=evaluate(model,val_loader)
    print(f"{label} KD → Val Top-1={top1:.2f}% | Top-5={top5:.2f}%")

print("\n--- Correlation Matrices ---")
corr_teacher=correlation_matrix(teacher,val_loader)
corr_soft=correlation_matrix(student_soft,val_loader)
corr_hard=correlation_matrix(student_hard,val_loader)
print("Mean |Teacher-Soft| diff:",np.abs(corr_teacher-corr_soft).mean())
print("Mean |Teacher-Hard| diff:",np.abs(corr_teacher-corr_hard).mean())

def tsne_summary(model, loader, n_samples=1000):
    model.eval(); X,Y=[],[]
    with torch.no_grad():
        for x,y in loader:
            X.append(model(x.to(device)).cpu()); Y.append(y)
            if len(torch.cat(Y))>=n_samples: break
    X=torch.cat(X)[:n_samples]; Y=torch.cat(Y)[:n_samples]
    tsne=TSNE(n_components=2,init='pca',random_state=0).fit_transform(X)
    print(f"t-SNE done: {tsne.shape}, labels {Y.unique().numel()} classes")
    return tsne,Y

print("\nRunning t-SNE (Soft student)...")
tsne_s,Ys=tsne_summary(student_soft,val_loader)
print("Running t-SNE (Hard student)...")
tsne_h,Yh=tsne_summary(student_hard,val_loader)

save_dir="/content/drive/MyDrive/KD_Experiments"
os.makedirs(save_dir,exist_ok=True)
torch.save(student_hard.state_dict(),os.path.join(save_dir,"KD_BASELINE_HARD.pth"))
torch.save(student_soft.state_dict(),os.path.join(save_dir,"KD_BASELINE_SOFT.pth"))
print(f"\nModels saved to {save_dir}")


LABEL SMOOTHING IMPLEMENTATION


In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from sklearn.manifold import TSNE
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------- IMAGE DATA ----------
# If full ImageNet is unavailable, use Tiny-ImageNet (≈1 GB)
train_dir = "/content/imagenet/train"
val_dir   = "/content/imagenet/val"

transform_train = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])
transform_val = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])

trainset = datasets.ImageFolder(train_dir, transform=transform_train)
valset   = datasets.ImageFolder(val_dir,   transform=transform_val)
train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
val_loader   = DataLoader(valset,   batch_size=64, shuffle=False, num_workers=4)
print(f"Train={len(trainset)}, Val={len(valset)}")

# ---------- MODELS ----------
teacher = models.vgg16(weights='IMAGENET1K_V1').to(device)
teacher.eval()
print("Teacher VGG-16 (ImageNet pretrained) loaded.")

student = models.vgg11(num_classes=1000).to(device)   # train from scratch
print("Student VGG-11 initialized (1000 classes).")


In [ ]:
def label_smoothing(targets, n_classes=1000, eps=0.1):
    """Return smoothed one-hot vectors"""
    with torch.no_grad():
        out = torch.full((targets.size(0), n_classes),
                         eps / (n_classes - 1), device=targets.device)
        out.scatter_(1, targets.unsqueeze(1), 1 - eps)
    return out

def kd_ls_loss(student_logits, teacher_logits, targets,
               T=4.0, alpha=0.7, eps=0.1):
    """Hinton KD + Szegedy Label Smoothing"""
    smoothed = label_smoothing(targets, student_logits.size(1), eps)
    log_s = F.log_softmax(student_logits / T, dim=1)
    soft_t = F.softmax(teacher_logits / T, dim=1)
    kl = F.kl_div(log_s, soft_t, reduction='batchmean') * (T*T)
    ce = -(smoothed * F.log_softmax(student_logits, dim=1)).sum(dim=1).mean()
    total = alpha * kl + (1 - alpha) * ce
    print(f"KL={kl.item():.4f} | CE={ce.item():.4f} | Total={total.item():.4f}")
    return total

def topk_acc(output, target, topk=(1,5)):
    maxk=max(topk)
    _,pred=output.topk(maxk,1,True,True)
    correct=pred.t().eq(target.view(1,-1).expand_as(pred))
    return [correct[:k].reshape(-1).float().sum(0).item() for k in topk]

def evaluate(model, loader):
    model.eval(); c1=c5=tot=0
    with torch.no_grad():
        for x,y in loader:
            x,y=x.to(device),y.to(device)
            out=model(x)
            r1,r5=topk_acc(out,y)
            c1+=r1; c5+=r5; tot+=y.size(0)
    return 100*c1/tot, 100*c5/tot

def run_training(epochs=20):
    opt=torch.optim.SGD(student.parameters(),lr=0.01,momentum=0.9,weight_decay=5e-4)
    for ep in range(1,epochs+1):
        student.train(); total,correct,loss_sum=0,0,0
        for x,y in tqdm(train_loader,desc=f"Epoch {ep}/{epochs}",leave=False):
            x,y=x.to(device),y.to(device)
            with torch.no_grad(): t_logits=teacher(x)
            s_logits=student(x)
            loss=kd_ls_loss(s_logits,t_logits,y,T=4.0,alpha=0.7,eps=0.1)
            opt.zero_grad(); loss.backward(); opt.step()
            loss_sum+=loss.item()*y.size(0)
            _,pred=s_logits.max(1)
            correct+=pred.eq(y).sum().item(); total+=y.size(0)
        train_acc=100*correct/total
        val1,val5=evaluate(student,val_loader)
        print(f"Epoch {ep:02d}: TrainAcc={train_acc:.2f}% | ValTop1={val1:.2f}% | ValTop5={val5:.2f}%")
    print("Training finished.")
    return student

student = run_training(epochs=20)


In [ ]:
def correlation_matrix(model, loader, n_batches=10):
    model.eval(); feats=[]
    with torch.no_grad():
        for i,(x,_) in enumerate(loader):
            if i>=n_batches: break
            feats.append(model(x.to(device)).cpu().numpy())
    feats=np.concatenate(feats,axis=0)
    return np.corrcoef(feats.T)

print("\n--- Final Evaluation ---")
train1,train5=evaluate(student,train_loader)
val1,val5=evaluate(student,val_loader)
print(f"Train: Top-1={train1:.2f}% | Top-5={train5:.2f}%")
print(f"Val:   Top-1={val1:.2f}% | Top-5={val5:.2f}%")

print("\n--- Correlation Matrices ---")
corr_teacher=correlation_matrix(teacher,val_loader)
corr_student=correlation_matrix(student,val_loader)
print("Mean |Teacher–Student| corr-diff:",
      np.abs(corr_teacher - corr_student).mean())

def tsne_summary(model, loader, n_samples=1000):
    model.eval(); X,Y=[],[]
    with torch.no_grad():
        for x,y in loader:
            X.append(model(x.to(device)).cpu()); Y.append(y)
            if len(torch.cat(Y))>=n_samples: break
    X=torch.cat(X)[:n_samples]; Y=torch.cat(Y)[:n_samples]
    tsne=TSNE(n_components=2,init='pca',random_state=0).fit_transform(X)
    print(f"t-SNE done: {tsne.shape}, {Y.unique().numel()} classes")
    return tsne,Y

print("\nRunning t-SNE (student)…")
tsne_out,Ys=tsne_summary(student,val_loader)

save_dir="/content/drive/MyDrive/LabelSmoothing_KD_ImageNet"
os.makedirs(save_dir,exist_ok=True)
torch.save(student.state_dict(), os.path.join(save_dir,"VGG11_LSR_KD.pth"))
print(f"\nModel saved to {save_dir}")
